In [20]:
import os
from datasets import Dataset, concatenate_datasets
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence

In [2]:
# 디바이스 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
print("CUDA 사용 가능 여부:", torch.cuda.is_available())

CUDA 사용 가능 여부: True


## 데이터 로드

r"C:\Users\MATH-3\Downloads\엔엘핑\train_data"

## LoRA 설정 및 모델 준비

## 파인튜닝

In [34]:
import os
from datasets import Dataset, concatenate_datasets
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

# 설정
train_data_dir = r"C:\Users\MATH-3\Downloads\엔엘핑\train_data"
model_name = "openai/whisper-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 로드 및 결합
def load_arrow_files(directory):
    dataset_list = []
    for file in tqdm(os.listdir(directory), desc="데이터 로드 중"):
        if file.endswith(".arrow"):
            file_path = os.path.join(directory, file)
            try:
                dataset = Dataset.from_file(file_path)
                dataset_list.append(dataset)
            except Exception as e:
                print(f"파일 로드 실패: {file_path}, 오류: {e}")
    if dataset_list:
        return concatenate_datasets(dataset_list)
    else:
        raise ValueError("유효한 데이터셋 파일이 없습니다.")

print("데이터 로드 중...")
train_dataset = load_arrow_files(train_data_dir)

# 필터링 전에 데이터셋 내용 확인
print(f"데이터셋 크기(필터링 전): {len(train_dataset)}")
print(f"샘플 데이터 확인: {train_dataset[0]}")  # 첫 번째 샘플 확인

# None 값 필터링
train_dataset = train_dataset.filter(lambda x: x["input_features"] is not None and x["labels"] is not None)

# 필터링 후 데이터셋 크기 확인
print(f"데이터셋 크기(필터링 후): {len(train_dataset)}")
if len(train_dataset) == 0:
    raise ValueError("필터링 결과 데이터셋이 비어 있습니다. 데이터를 확인하세요.")


데이터 로드 중...


데이터 로드 중: 100%|██████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 39.23it/s]


데이터셋 크기(필터링 전): 49276
샘플 데이터 확인: {'indices': 16256, 'start_time': None, 'end_time': None, 'input_features': None, 'labels': None}


Filter: 100%|█████████████████████████████████████████████████████████████| 49276/49276 [21:29<00:00, 38.21 examples/s]

데이터셋 크기(필터링 후): 16764


In [35]:
from torch.nn.utils.rnn import pad_sequence

# Whisper 모델 및 프로세서 준비
processor = WhisperProcessor.from_pretrained(model_name)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    #task_type="SEQ_2_SEQ_LM"
)

# Whisper 모델 로드 및 LoRA 적용
print("모델 준비 중...")
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model = get_peft_model(model, lora_config)  # LoRA 적용
model.to(device)

# DataLoader 생성
MAX_LABEL_LENGTH = 60  # 고정된 최대 길이

def collate_fn(batch):
    input_features = [torch.tensor(item["input_features"]) for item in batch]
    labels = [torch.tensor(item["labels"]) for item in batch]

    # 입력 데이터 패딩
    input_features = pad_sequence(input_features, batch_first=True, padding_value=0.0)

    # 라벨 패딩 - 고정된 MAX_LABEL_LENGTH 적용
    labels = pad_sequence(
        labels,
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )

    # 라벨 길이를 고정된 MAX_LABEL_LENGTH로 자르거나 패딩
    if labels.shape[1] < MAX_LABEL_LENGTH:
        padding = torch.full(
            (labels.shape[0], MAX_LABEL_LENGTH - labels.shape[1]),
            processor.tokenizer.pad_token_id,
        )
        labels = torch.cat([labels, padding], dim=1)
    else:
        labels = labels[:, :MAX_LABEL_LENGTH]

    return {"input_features": input_features, "labels": labels}


train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

# 모델 학습 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()

# 모델 학습 루프
print("모델 학습 시작...")
for epoch in range(3):
    print(f"\nEpoch {epoch + 1} 시작...")
    epoch_loss = 0

    with tqdm(train_dataloader, desc=f"Epoch {epoch + 1} 진행 중", leave=True, dynamic_ncols=True) as progress_bar:
        for batch in progress_bar:
            # 배치 데이터 추출
            input_features = batch["input_features"].to(device)
            labels = batch["labels"].to(device)

            # Whisper 모델 호출
            outputs = model(input_features=input_features, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            epoch_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})  # Loss 표시

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} 완료, 평균 Loss: {avg_loss:.4f}")

# 모델 저장
output_dir = "lora_whisper_model"
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

print(f"모델 저장 완료: {output_dir}")


모델 준비 중...
모델 학습 시작...

Epoch 1 시작...


Epoch 1 진행 중: 100%|████████████████████████████████████████████████| 2096/2096 [39:15<00:00,  1.12s/it, Loss=1.5715]


Epoch 1 완료, 평균 Loss: 1.9961

Epoch 2 시작...


Epoch 2 진행 중: 100%|████████████████████████████████████████████████| 2096/2096 [31:27<00:00,  1.11it/s, Loss=1.7544]


Epoch 2 완료, 평균 Loss: 1.8435

Epoch 3 시작...


Epoch 3 진행 중: 100%|████████████████████████████████████████████████| 2096/2096 [31:12<00:00,  1.12it/s, Loss=1.0675]


Epoch 3 완료, 평균 Loss: 1.8115
모델 저장 완료: lora_whisper_model


In [33]:
"""from torch.nn.utils.rnn import pad_sequence

# Whisper 모델 및 프로세서 준비
processor = WhisperProcessor.from_pretrained(model_name)

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    #task_type="SEQ_2_SEQ_LM"
)

# Whisper 모델 로드 및 LoRA 적용
print("모델 준비 중...")
model = WhisperForConditionalGeneration.from_pretrained(model_name)
model = get_peft_model(model, lora_config)  # LoRA 적용
model.to(device)

# DataLoader 생성
# 고정된 최대 길이 설정 (예: 60)
MAX_LABEL_LENGTH = 60

def collate_fn(batch):
    input_features = [torch.tensor(item["input_features"]) for item in batch]
    labels = [torch.tensor(item["labels"]) for item in batch]

    # 입력 데이터 패딩
    input_features = pad_sequence(input_features, batch_first=True, padding_value=0.0)

    # 라벨 패딩 - 고정된 MAX_LABEL_LENGTH 적용
    labels = pad_sequence(
        labels,
        batch_first=True,
        padding_value=processor.tokenizer.pad_token_id
    )

    # 라벨 길이를 고정된 MAX_LABEL_LENGTH로 자르거나 패딩
    if labels.shape[1] < MAX_LABEL_LENGTH:
        padding = torch.full(
            (labels.shape[0], MAX_LABEL_LENGTH - labels.shape[1]),
            processor.tokenizer.pad_token_id,
        )
        labels = torch.cat([labels, padding], dim=1)
    else:
        labels = labels[:, :MAX_LABEL_LENGTH]

    return {"input_features": input_features, "labels": labels}


train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=collate_fn, shuffle=True)

# 모델 학습 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()

# 모델 학습 루프
print("모델 학습 시작...")
for epoch in range(3):
    print(f"Epoch {epoch + 1} 시작...")
    epoch_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1} 진행 중"):
        # 배치 데이터 추출
        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        # 디버깅: 입력 데이터 확인
        print(f"Input features shape: {input_features.shape}")
        print(f"Labels shape: {labels.shape}")

        # Whisper 모델 호출
        try:
            outputs = model(input_features=input_features, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            epoch_loss += loss.item()
        except TypeError as e:
            print(f"오류 발생: {e}")
            print("입력 데이터 확인 중...")
            print(f"Input features: {input_features}")
            print(f"Labels: {labels}")
            raise e

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1} 완료, 평균 Loss: {avg_loss:.4f}")


# 모델 저장
output_dir = "lora_whisper_model"
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

print(f"모델 저장 완료: {output_dir}")
"""

모델 준비 중...
모델 학습 시작...
Epoch 1 시작...


Epoch 1 진행 중:   0%|                                                                        | 0/2032 [00:00<?, ?it/s]Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Input features shape: torch.Size([8, 80, 3000])
Labels shape: torch.Size([8, 60])


Epoch 1 진행 중:   0%|                                                                | 1/2032 [00:01<47:56,  1.42s/it]

Input features shape: torch.Size([8, 80, 3000])
Labels shape: torch.Size([8, 60])


...

Input features shape: torch.Size([8, 80, 3000])
Labels shape: torch.Size([8, 60])


Epoch 1 진행 중:  14%|████████▌                                                     | 280/2032 [05:08<31:57,  1.09s/it]

Input features shape: torch.Size([8, 80, 3000])
Labels shape: torch.Size([8, 60])


Epoch 1 진행 중:  14%|████████▌                                                     | 281/2032 [05:10<32:14,  1.10s/it]


KeyboardInterrupt: 

In [31]:
# Whisper 모델 입력 확인 모델에 전달하는 input_features와 labels의 크기를 출력해 예상과 일치하는지 확인하세요
print(f"Input features shape: {batch['input_features'].shape}")
print(f"Labels shape: {batch['labels'].shape}")


Input features shape: torch.Size([8, 80, 3000])
Labels shape: torch.Size([8, 60])


In [27]:
#학습 루프에서 데이터 확인: 학습 루프 내부에서 입력 데이터와 레이블의 크기가 모델에 맞게 처리되고 있는지 확인합니다.

for batch in train_dataloader:
    print(f"Input features shape: {batch['input_features'].shape}")
    print(f"Labels shape: {batch['labels'].shape}")
    break  # 첫 배치만 확인


Input features shape: torch.Size([8, 80, 3000])
Labels shape: torch.Size([8, 60])


## 데이터 확인

In [27]:
import os

directory = r"C:\Users\MATH-3\Downloads\엔엘핑\train_data"
for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    print(f"{file}: {os.path.getsize(file_path)} bytes")


cache-3b97b2893613e14f.arrow: 304 bytes
cache-3ee629c5e9a0e2d9.arrow: 3721072 bytes
data-00000-of-00033.arrow: 487961424 bytes
data-00001-of-00033.arrow: 487958232 bytes
data-00002-of-00033.arrow: 487958952 bytes
data-00003-of-00033.arrow: 487962256 bytes
data-00004-of-00033.arrow: 487962072 bytes
data-00005-of-00033.arrow: 487959824 bytes
data-00006-of-00033.arrow: 487961104 bytes
data-00007-of-00033.arrow: 487959856 bytes
data-00009-of-00033.arrow: 487959736 bytes
data-00010-of-00033.arrow: 487960336 bytes
data-00011-of-00033.arrow: 487959704 bytes
data-00012-of-00033.arrow: 487957792 bytes
data-00013-of-00033.arrow: 487959568 bytes
data-00014-of-00033.arrow: 487962776 bytes
data-00015-of-00033.arrow: 487957584 bytes
data-00016-of-00033.arrow: 487959576 bytes
data-00017-of-00033.arrow: 487959488 bytes
data-00018-of-00033.arrow: 487957080 bytes
data-00019-of-00033.arrow: 487959664 bytes
data-00020-of-00033.arrow: 487959896 bytes
data-00021-of-00033.arrow: 487959992 bytes
data-00022-of

In [32]:
import os
from datasets import Dataset

# 폴더 경로 설정
directory = r"C:\Users\MATH-3\Downloads\엔엘핑\train_data"

# 폴더 내 모든 .arrow 파일 확인
for file_name in os.listdir(directory):
    if file_name.endswith(".arrow"):
        file_path = os.path.join(directory, file_name)
        try:
            dataset = Dataset.from_file(file_path)
            print(f"파일 로드 성공: {file_name}")
            print(f"샘플 데이터: {dataset[0]}")  # 첫 번째 샘플만 출력
            print(f"데이터셋 크기: {len(dataset)}")
        except Exception as e:
            print(f"파일 로드 실패: {file_name}, 오류: {e}")


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

